# Imports

In [1]:
import os
import pathlib
import tensorflow as tf
import glob
import cv2
import time
import numpy as np
from mean_average_precision import MetricBuilder
import xml.etree.cElementTree as ET
import numpy as np
from PIL import Image
import warnings

# Paths/Variables

In [2]:
PATH_TO_MODEL_DIR = 'exported-model_ssd_fpn'
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
PATH_TO_LABELS = 'label_map_licence.txt'
IMAGE_PATHS = "./Input/*.jpg"
RESULT_DIR = "./Output"
os.makedirs(RESULT_DIR,exist_ok=True)

THRESHOLD = 0.4

In [3]:
##Function to extract classes from labelfile
def create_category_index(label_path):
    f = open(label_path)
    category_index = {}
    for i, val in enumerate(f):
        if i != 0:
            val = val[:-1]
            if val != '???':
                category_index.update({(i-1): {'id': (i-1), 'name': val}})
            
    f.close()
    #print(category_index)
    # exit()
    return category_index
id_to_cls = create_category_index(PATH_TO_LABELS)
id_to_cls

{0: {'id': 0, 'name': 'license_plate'}}

# Load SSD-Mobilenet-fpnlite tensorflow model

In [4]:
print('Loading model...', end='')
start_time = time.time()
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)
end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 13.166779041290283 seconds


# Get Ground Truth to calculate mean-average-precision

In [5]:
def calculate_ground_truth(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    bboxes = []   ##ground Truth  [xmin, ymin, xmax, ymax, class_id, difficult, crowd]
    for member in root.findall('object'):
        class_id = 0 #class_to_id[member[0].text]
        bboxes.append([int(member[4][0].text), int(member[4][1].text), int(member[4][2].text), int(member[4][3].text), class_id,0,0])
    return np.array(bboxes)

# NMS

In [6]:
def nms(dets_tup, thresh=0.5):
    if len(dets_tup)<=1:
        return dets_tup
    dets = np.asarray(dets_tup)
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]

    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    order = y2.argsort()#[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(dets_tup[i])
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)

        inter = w * h
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        inds = np.where(ovr <= thresh)[0]
        order = order[inds + 1]
        
    return keep


# Prediction on image

In [7]:
def predict_tf(image_np):
    h,w = image_np.shape[:2]

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    start_time = time.time()
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    preds = [] #[xmin, ymin, xmax, ymax, class_id, confidence]
    for rec,cls_,sc in zip(detections['detection_boxes'],detections['detection_classes'],detections['detection_scores']):
        # print(rec,cls_,sc)
        # print(clss_txt)
        if sc>THRESHOLD:
            preds.append([int(rec[1]*w), int(rec[0]*h), int(rec[3]*w), int(rec[2]*h),cls_-1,sc])
            
    print("INFERENCE TIME: {} ms".format((time.time()-start_time)*1000))
    return np.array(nms(preds))

# Iterate over InputDir

In [9]:
sst = time.time()
metric_fn = MetricBuilder.build_evaluation_metric("map_2d", async_mode=True, num_classes=1)
ind=0
for image_path in glob.glob(IMAGE_PATHS):
    
    fn = image_path.split("/")[-1]

    if ind%100==0:
        print("Done: {}, Total Time taken to Read/Process/Inference/Write: {}".format(ind,time.time()-sst))
        print(f"VOC PASCAL mAP in till {ind} points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")

    xml_path = image_path.replace("Input/","InputAnnotations/").replace(".jpg",".xml")
    gt = calculate_ground_truth(xml_path)

    image_np = cv2.imread(image_path)
    preds = predict_tf(image_np)
    for rec in preds:
        cv2.rectangle(image_np, (int(rec[0]), int(rec[1])), (int(rec[2]), int(rec[3])),(255,0, 12), 2)
        cls_sc_txt = id_to_cls[rec[4]]['name']+"_"+str(round(rec[5]*100,2))
        
    metric_fn.add(preds, gt)
    
    ind+=1

    cv2.imwrite(RESULT_DIR+"/"+fn,image_np)
    
print("Done: {}, Total Time taken to Read/Process/Inference/Write: {}".format(ind,time.time()-sst))
print("Avg FPS: ",ind/(time.time()-sst))
print(f"mAP in till {ind} points: {metric_fn.value(iou_thresholds=0.5)['mAP']}")

Done: 0, Total Time taken to Read/Process/Inference/Write: 0.07486653327941895
VOC PASCAL mAP in till 0 points: 0.0
INFERENCE TIME: 84.33222770690918 ms
INFERENCE TIME: 93.43075752258301 ms
INFERENCE TIME: 71.79713249206543 ms
INFERENCE TIME: 60.846805572509766 ms
INFERENCE TIME: 77.02207565307617 ms
INFERENCE TIME: 74.25355911254883 ms
INFERENCE TIME: 72.53646850585938 ms
INFERENCE TIME: 89.48373794555664 ms
INFERENCE TIME: 67.413330078125 ms
INFERENCE TIME: 63.04574012756348 ms
INFERENCE TIME: 64.07594680786133 ms
INFERENCE TIME: 71.5646743774414 ms
INFERENCE TIME: 93.20902824401855 ms
INFERENCE TIME: 78.25112342834473 ms
INFERENCE TIME: 105.68881034851074 ms
INFERENCE TIME: 57.32464790344238 ms
INFERENCE TIME: 63.72809410095215 ms
INFERENCE TIME: 145.5230712890625 ms
INFERENCE TIME: 150.22754669189453 ms
INFERENCE TIME: 65.30356407165527 ms
INFERENCE TIME: 73.99582862854004 ms
INFERENCE TIME: 64.92996215820312 ms
INFERENCE TIME: 122.31588363647461 ms
INFERENCE TIME: 128.021955490112